In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Lecture des données

In [6]:
transactions_101_104 = pd.read_csv('../raw_data/transaction_train_week_101_104.csv')
#101= dernière semaine d'aout
#102= première semaine septembre
#103= deuxième semaine septembre
#104= troisème semaine de septembre (qui nous servira de target dans le cadre du projet)

In [58]:
transactions_101_104.head()

,Unnamed: 0,Unnamed: 0.1,t_dat,customer_id,article_id,price,sales_channel_id
0,0,30665422,101,0000e9a4db2da4e9c68558ad1e7df570d02767b213ec6b...,880553001,0.025407,2
1,1,30665423,101,0000e9a4db2da4e9c68558ad1e7df570d02767b213ec6b...,880553001,0.025407,2
2,2,30665424,101,00047d50bde8751b26aaec7536ccc5fd92828ffa0a396d...,909081004,0.033881,2
3,3,30665425,101,00047d50bde8751b26aaec7536ccc5fd92828ffa0a396d...,516903024,0.025407,2
4,4,30665426,101,00047d50bde8751b26aaec7536ccc5fd92828ffa0a396d...,897679003,0.042356,2


In [7]:
# note du 06042022: à ce stade de l'analyse les customers sans information sur l'âge ne sont pas inclus
S1= pd.read_csv('../raw_data/customers_age_16_17.csv')
S2= pd.read_csv('../raw_data/customers_age_18_19.csv')
S3= pd.read_csv('../raw_data/customers_age_20_21.csv')
S4= pd.read_csv('../raw_data/customers_age_22_23.csv')
S5= pd.read_csv('../raw_data/customers_age_24_25.csv')
S6= pd.read_csv('../raw_data/customers_age_26_27.csv')
S7= pd.read_csv('../raw_data/customers_age_28_29.csv')
S8= pd.read_csv('../raw_data/customers_age_30_34.csv')
S9= pd.read_csv('../raw_data/customers_age_35_39.csv')
S10= pd.read_csv('../raw_data/customers_age_40_44.csv')
S11= pd.read_csv('../raw_data/customers_age_45_49.csv')
S12= pd.read_csv('../raw_data/customers_age_50_54.csv')
S13= pd.read_csv('../raw_data/customers_age_55_59.csv')
S14= pd.read_csv('../raw_data/customers_age_60.csv')

In [8]:
S1.shape, S2.shape, S3.shape, S4.shape, S5.shape, S6.shape, S7.shape, S8.shape, S9.shape, S10.shape, S11.shape, S12.shape, S13.shape, S14.shape

((9553, 8),
 (62030, 8),
 (122726, 8),
 (106736, 8),
 (111113, 8),
 (102792, 8),
 (84991, 8),
 (150520, 8),
 (83548, 8),
 (85493, 8),
 (118625, 8),
 (135326, 8),
 (90916, 8),
 (91750, 8))

In [83]:
S1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9553 entries, 0 to 9552
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   customer_id  9553 non-null   object
 1   age_id       9553 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 149.4+ KB


# Préparation du dataset Transactions

In [9]:
# je garde uniquement la week, le customer id pour le merge, et l'article ID pour récupérer les plus populaires:
transactions_101_104=transactions_101_104.drop(columns=['Unnamed: 0', 'Unnamed: 0.1', 'price', 'sales_channel_id'])

In [10]:
transactions_101_104.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1057905 entries, 0 to 1057904
Data columns (total 3 columns):
 #   Column       Non-Null Count    Dtype 
---  ------       --------------    ----- 
 0   t_dat        1057905 non-null  int64 
 1   customer_id  1057905 non-null  object
 2   article_id   1057905 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 24.2+ MB


In [11]:
# je crée des dummies par week si jamais plus tard on a besoin de baselines pour d'autres semaines que la 104
article_history=transactions_101_104
article_history['in101']=article_history['t_dat'].apply(lambda x: 1 if x==101 else 0)
article_history['in102']=article_history['t_dat'].apply(lambda x: 1 if x==102 else 0)
article_history['in103']=article_history['t_dat'].apply(lambda x: 1 if x==103 else 0)
article_history['in104']=article_history['t_dat'].apply(lambda x: 1 if x==104 else 0)
article_history

,t_dat,customer_id,article_id,in101,in102,in103,in104
0,101,0000e9a4db2da4e9c68558ad1e7df570d02767b213ec6b...,880553001,1,0,0,0
1,101,0000e9a4db2da4e9c68558ad1e7df570d02767b213ec6b...,880553001,1,0,0,0
2,101,00047d50bde8751b26aaec7536ccc5fd92828ffa0a396d...,909081004,1,0,0,0
3,101,00047d50bde8751b26aaec7536ccc5fd92828ffa0a396d...,516903024,1,0,0,0
4,101,00047d50bde8751b26aaec7536ccc5fd92828ffa0a396d...,897679003,1,0,0,0
...,...,...,...,...,...,...,...
1057900,104,fff4381593e170ca0aea188998487c830d9a4070c9ec4b...,799365001,0,0,0,1
1057901,104,fff7e7674509592818bf453391af43a85eaaac9a52d858...,624486049,0,0,0,1
1057902,104,fff871bf24b40fd1290215414d760afaa69bb164d2b970...,717490010,0,0,0,1
1057903,104,fff871bf24b40fd1290215414d760afaa69bb164d2b970...,717490058,0,0,0,1


# merge des données segments aux données transactions

In [92]:
S1_achats=S1[['customer_id','age_id']].merge(article_history)

In [93]:
S1_achats.shape #(5955, 8)

(5955, 8)

In [95]:
S1_achats.head()

,customer_id,age_id,t_dat,article_id,in101,in102,in103,in104
0,0006ca2c5ecdbdcc1de5e72adffc6c33448fc2aaa62afe...,0,101,572998001,1,0,0,0
1,0006ca2c5ecdbdcc1de5e72adffc6c33448fc2aaa62afe...,0,101,572998009,1,0,0,0
2,00089f13f465ec902e5c49a3bb408c5e31205096d6f267...,0,101,926502001,1,0,0,0
3,000e3f587242eb077685a487ad27dad632a4801576dfd1...,0,101,501620049,1,0,0,0
4,001c42459c8978d55a69fa1528975893b011a6539631f3...,0,101,885077003,1,0,0,0


In [96]:
S1_articles=S1_achats.groupby(['article_id']).sum()
S1_articles.shape

(2501, 6)

In [103]:
S1_Top12_104=S1_articles.sort_values(by='in104', ascending=False).head(12)
S1_Top12_104

,age_id,t_dat,in101,in102,in103,in104
article_id,,,,,,
448509014,0,3723,4,2,5,25
797988002,0,3303,7,2,0,23
685814003,0,3725,1,0,16,19
715624052,0,2173,3,0,2,16
682550002,0,2282,0,0,6,16
456163086,0,2380,2,1,4,16
730683050,0,2269,5,2,0,15
715624001,0,3494,10,2,8,14
871517008,0,2171,3,0,4,14


In [20]:
def Baseline_0 (segment,transact_history,week): # exemple: week104name ='in104' dans l'historique des transactions
    
    week_dummy="in" + f'{week}'
    achats=segment[['customer_id']].merge(transact_history[['customer_id','article_id',week_dummy]])
    articles= achats.groupby(['article_id']).sum()
    Top12_arr=articles.sort_values(by=week_dummy, ascending=False).head(12)
    Top12=""
    for i in range (0, 12):
        if i==0 :
            Top12=str(Top12_arr.index[i])
        else: 
            Top12=Top12 + " "+ str(Top12_arr.index[i])
            
    #je donne à tous mes customers de mon segment le baseline top12 calculée ci-dessus
    segment['Baseline0']=Top12
    
    #Ancien code ici *******************************************************************
    #je retourne sur leurs achats et je vérifie si l'articleID acheté est dans la TOP12 (1 si oui, O sinon)
    #achats['inBaseline0']=achats['article_id'].apply(lambda x: 1 if str(x) in Top12 else 0)
    #Baseline0_precision=achats.groupby(['customer_id']).count().drop(columns= ['article_id',week_dummy] ) 
    
    #penser à faire un changement de nom de col pour la précision
    #segment_BS=segment.merge(Baseline0_precision, how = 'left', on='customer_id') 
    #  *******************************************************************
    
    #je retourne sur leurs achats et je vérifie si l'articleID acheté est dans la TOP12 (1 si oui, O sinon)
    achats['inBaseline0']=achats['article_id'].apply(lambda x: 1 if str(x) in Top12 else 0)
    Baseline0_precision=achats.groupby(['customer_id']).count().drop(columns= ['article_id',week_dummy]).rename(columns={'inBaseline0':'Precision'})
    print("Statistiques concernant le segment:")
    print("Taille du segment : " + f'{len(segment)}')
    print("Nbre d'acheteurs : " + f'{Baseline0_precision.count()}')
    print("Nbre de ref_achetées : " + f'{articles.count()}')
    print("Précision moyenne:" + f'{Baseline0_precision.mean()}')
    
    segment_BS=segment.merge(Baseline0_precision, how = 'left', on='customer_id') 
    #return segment_BS
    return "fin de traitement"
  

In [21]:
Baseline_0(S1,article_history,'104')

Statistiques concernant le segment:
Taille du segment : 9553
Nbre d'acheteurs : Precision    2073
dtype: int64
Nbre de ref_achetées : in104    2501
dtype: int64
Précision moyenne:Precision    2.872648
dtype: float64


'fin de traitement'

In [22]:
Baseline_0(S2,article_history,'104')

Statistiques concernant le segment:
Taille du segment : 62030
Nbre d'acheteurs : Precision    10420
dtype: int64
Nbre de ref_achetées : in104    7107
dtype: int64
Précision moyenne:Precision    3.071689
dtype: float64


'fin de traitement'

In [23]:
Baseline_0(S3,article_history,'104')

Statistiques concernant le segment:
Taille du segment : 122726
Nbre d'acheteurs : Precision    22157
dtype: int64
Nbre de ref_achetées : in104    10879
dtype: int64
Précision moyenne:Precision    3.512479
dtype: float64


'fin de traitement'

In [24]:
Baseline_0(S4,article_history,'104')

Statistiques concernant le segment:
Taille du segment : 106736
Nbre d'acheteurs : Precision    21268
dtype: int64
Nbre de ref_achetées : in104    12163
dtype: int64
Précision moyenne:Precision    4.252398
dtype: float64


'fin de traitement'

In [25]:
Baseline_0(S5,article_history,'104')

Statistiques concernant le segment:
Taille du segment : 111113
Nbre d'acheteurs : Precision    22596
dtype: int64
Nbre de ref_achetées : in104    13439
dtype: int64
Précision moyenne:Precision    4.686582
dtype: float64


'fin de traitement'

In [26]:
Baseline_0(S6,article_history,'104')

Statistiques concernant le segment:
Taille du segment : 102792
Nbre d'acheteurs : Precision    20006
dtype: int64
Nbre de ref_achetées : in104    13627
dtype: int64
Précision moyenne:Precision    4.963361
dtype: float64


'fin de traitement'

In [27]:
Baseline_0(S7,article_history,'104')

Statistiques concernant le segment:
Taille du segment : 84991
Nbre d'acheteurs : Precision    15646
dtype: int64
Nbre de ref_achetées : in104    13178
dtype: int64
Précision moyenne:Precision    5.257638
dtype: float64


'fin de traitement'

In [28]:
Baseline_0(S8,article_history,'104')

Statistiques concernant le segment:
Taille du segment : 150520
Nbre d'acheteurs : Precision    25500
dtype: int64
Nbre de ref_achetées : in104    16744
dtype: int64
Précision moyenne:Precision    5.34349
dtype: float64


'fin de traitement'

In [29]:
Baseline_0(S9,article_history,'104')

Statistiques concernant le segment:
Taille du segment : 83548
Nbre d'acheteurs : Precision    11969
dtype: int64
Nbre de ref_achetées : in104    13453
dtype: int64
Précision moyenne:Precision    5.272872
dtype: float64


'fin de traitement'

In [30]:
Baseline_0(S10,article_history,'104')

Statistiques concernant le segment:
Taille du segment : 85493
Nbre d'acheteurs : Precision    12787
dtype: int64
Nbre de ref_achetées : in104    12635
dtype: int64
Précision moyenne:Precision    4.853914
dtype: float64


'fin de traitement'

In [31]:
Baseline_0(S11,article_history,'104')

Statistiques concernant le segment:
Taille du segment : 118625
Nbre d'acheteurs : Precision    19717
dtype: int64
Nbre de ref_achetées : in104    13538
dtype: int64
Précision moyenne:Precision    4.638028
dtype: float64


'fin de traitement'

In [32]:
Baseline_0(S12,article_history,'104')

Statistiques concernant le segment:
Taille du segment : 135326
Nbre d'acheteurs : Precision    22731
dtype: int64
Nbre de ref_achetées : in104    13362
dtype: int64
Précision moyenne:Precision    4.377238
dtype: float64


'fin de traitement'

In [33]:
Baseline_0(S13,article_history,'104')

Statistiques concernant le segment:
Taille du segment : 90916
Nbre d'acheteurs : Precision    14446
dtype: int64
Nbre de ref_achetées : in104    11122
dtype: int64
Précision moyenne:Precision    4.15333
dtype: float64


'fin de traitement'

In [34]:
Baseline_0(S14,article_history,'104')

Statistiques concernant le segment:
Taille du segment : 91750
Nbre d'acheteurs : Precision    13022
dtype: int64
Nbre de ref_achetées : in104    9732
dtype: int64
Précision moyenne:Precision    3.675779
dtype: float64


'fin de traitement'